In [32]:
import cv2
import os

import numpy as np

app_dir = '/Users/nikhilvs/repos/nyu/birds-eye-view-predictor'

In [75]:
def convert_map_to_road_map(ego_map):
    mask = (ego_map[0,:,:] == 1) * (ego_map[1,:,:] == 1) * (ego_map[2,:,:] == 1)
    # mask = (ego_map[:,:, 0] == 255) * (ego_map[:,:, 1] == 255) * (ego_map[:,:, 2] == 255)

    return (~mask)

import scipy.misc


def generate_scene_video(scene):
    all_array = []

    samples = list(range(126))
    image_dir = os.path.join(app_dir, 'artifacts/data/camera_data')
    video_dir = os.path.join(app_dir, 'artifacts/videos')
    size = ()

    for sample in samples:
        sample_dir = os.path.join(image_dir, 'scene_' + str(scene), 'sample_' + str(sample))
        image_front = cv2.imread(os.path.join(sample_dir, 'CAM_FRONT.jpeg'))
        image_front_left = cv2.imread(os.path.join(sample_dir, 'CAM_FRONT_LEFT.jpeg'))
        image_front_right = cv2.imread(os.path.join(sample_dir, 'CAM_FRONT_RIGHT.jpeg'))
        image_back = cv2.imread(os.path.join(sample_dir, 'CAM_BACK.jpeg'))
        image_back_left = cv2.imread(os.path.join(sample_dir, 'CAM_BACK_LEFT.jpeg'))
        image_back_right = cv2.imread(os.path.join(sample_dir, 'CAM_BACK_RIGHT.jpeg'))
        image_ego = cv2.imread(os.path.join(sample_dir, 'ego.png'))
        
        # image_ego = convert_map_to_road_map(image_ego)

        h, w, c = image_front.shape
        size = (int(3 * w / 2), int(2 * h / 2))
        full_image_size = (int(3 * w), int(2 * h / 2))

        front_all = np.concatenate((image_front_left, image_front, image_front_right), axis = 1)
        back_all = np.concatenate((image_back_left, image_back, image_back_right), axis = 1)

        image_all = np.concatenate((front_all, back_all), axis = 0)
        image_all = cv2.resize(image_all, size)
        image_ego = cv2.resize(image_ego, size)

        image_full = np.concatenate((image_all, image_ego), axis = 1)
        all_array.append(image_full)

    video_path = os.path.join(video_dir, f'scene_{scene}.avi')
    out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'DIVX'), 15, full_image_size)

    for i in range(len(all_array)):
        out.write(all_array[i])
    out.release()

In [76]:
generate_scene_video(123)


In [35]:
def generate_video_by_directories(video_name, dir1, dir2, dir3):
    app_dir = '/Users/nikhilvs/repos/nyu/birds-eye-view-predictor'
    
    dir1_files = sorted(list(os.listdir(dir1)))
    dir2_files = sorted(list(os.listdir(dir2)))
    dir3_files = sorted(list(os.listdir(dir3)))
    # dir4_files = sorted(list(os.listdir(dir4)))

    all_array = []
    video_dir = os.path.join(app_dir, 'artifacts/videos')
    for i in range(len(dir1_files)):
        image1 = cv2.imread(os.path.join(dir1, dir1_files[i]))
        image2 = cv2.imread(os.path.join(dir2, dir2_files[i]))
        image3 = cv2.imread(os.path.join(dir3, dir3_files[i]))
        # image4 = cv2.imread(os.path.join(dir4, dir4_files[i]))
        image1 = cv2.resize(image1, (256, 256))
        image2 = cv2.resize(image2, (256, 256))
        image3 = cv2.resize(image3, (256, 256))
        # image4 = cv2.resize(image4, (256, 256))

        h, w, c = image1.shape
        size = int(w), int(h)
        full_image_size = (3 * int(w), int(h))

        images_all = np.concatenate((image1, image2, image3), axis = 1)
        all_array.append(images_all)
    
    video_path = os.path.join(video_dir, video_name)
    out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'DIVX'), 1, full_image_size)

    for i in range(len(all_array)):
        out.write(all_array[i])
    out.release()


In [30]:
generate_video_by_directories(
    '/Users/nikhilvs/repos/nyu/birds-eye-view-predictor/artifacts/data/pretrained_resnet_video_images/original_input',
    '/Users/nikhilvs/repos/nyu/birds-eye-view-predictor/artifacts/data/pretrained_resnet_video_images/original_output',
    '/Users/nikhilvs/repos/nyu/birds-eye-view-predictor/artifacts/data/pretrained_resnet_video_images/predicted_output',
    '/Users/nikhilvs/repos/nyu/birds-eye-view-predictor/artifacts/data/pretrained_resnet_video_images/predicted_output_2'
)


In [36]:
views = ['back', 'back_left', 'back_right', 'front', 'front_left', 'front_right']
epochs = [170, 300, 295, 400, 250, 250]
mono_dir = '/Users/nikhilvs/repos/nyu/birds-eye-view-predictor/artifacts/mono'
image_dir = os.path.join(app_dir, 'artifacts/data/camera_data')

for i in range(len(views)):
    view = views[i]
    epoch = epochs[i]

    input_dir = os.path.join(mono_dir, 'inputs', view)
    expected_output_dir = os.path.join(mono_dir, 'expected_outputs', view)
    output_dir = os.path.join(mono_dir, 'outputs', view + '_' + str(epoch))

    generate_video_by_directories(
        view + '_e' + str(epoch) + '.avi',
        input_dir,
        expected_output_dir, 
        output_dir
    )


In [53]:
app_dir = '/Users/nikhilvs/repos/nyu/birds-eye-view-predictor'
camera_dir = '/Users/nikhilvs/repos/nyu/birds-eye-view-predictor/artifacts/data/camera_data'
expected_dir = '/Users/nikhilvs/Downloads/bounding_box_top_view/target_rm_target_bb'
predicted_dir = '/Users/nikhilvs/Downloads/bounding_box_top_view/target_rm_predicted_bb'
all_array = []

for scene in [132]:
    for sample in range(126):
        camera_image_dir = os.path.join(camera_dir, 'scene_' + str(scene), 'sample_' + str(sample))
        image_front = cv2.imread(os.path.join(camera_image_dir, 'CAM_FRONT.jpeg'))
        image_front_left = cv2.imread(os.path.join(camera_image_dir, 'CAM_FRONT_LEFT.jpeg'))
        image_front_right = cv2.imread(os.path.join(camera_image_dir, 'CAM_FRONT_RIGHT.jpeg'))
        image_back = cv2.imread(os.path.join(camera_image_dir, 'CAM_BACK.jpeg'))
        image_back_left = cv2.imread(os.path.join(camera_image_dir, 'CAM_BACK_LEFT.jpeg'))
        image_back_right = cv2.imread(os.path.join(camera_image_dir, 'CAM_BACK_RIGHT.jpeg'))

        image_front = cv2.resize(image_front, (200, 200))
        image_front_left = cv2.resize(image_front_left, (200, 200))
        image_front_right = cv2.resize(image_front_right, (200, 200))
        image_back = cv2.resize(image_back, (200, 200))
        image_back_left = cv2.resize(image_back_left, (200, 200))
        image_back_right = cv2.resize(image_back_right, (200, 200))
        
        expected_image_path = os.path.join(expected_dir, str(scene) + '_' + str(sample) + '.png')
        predicted_image_path = os.path.join(predicted_dir, str(scene) + '_' + str(sample) + '.png')
        expected_image = cv2.imread(expected_image_path)
        predicted_image = cv2.imread(predicted_image_path)

        expected_image = cv2.resize(expected_image, (200, 200))
        predicted_image = cv2.resize(predicted_image, (200, 200))

        # h, w, c = image_front.shape
        # size = (int(3 * w / 2), int(2 * h / 2))
        full_image_size = (200 * 4, 200 * 2)

        front_all = np.concatenate((image_front_left, image_front, image_front_right, expected_image), axis = 1)
        back_all = np.concatenate((image_back_left, image_back, image_back_right, predicted_image), axis = 1)

        image_all = np.concatenate((front_all, back_all), axis = 0)
        # image_all = cv2.resize(image_all, size)

        # image_full = np.concatenate((image_all, image_ego), axis = 1)
        all_array.append(image_all)


video_dir = os.path.join(app_dir, 'artifacts/videos')
video_path = os.path.join(video_dir, 'final_road_map.mpeg')
out = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'DIVX'), 15, full_image_size)

for i in range(len(all_array)):
    out.write(all_array[i])
out.release()